In [ ]:
import pandas as pd

churn_cust_info = pd.read_excel("Bank+Customer+Churn/Bank_Churn_Messy.xlsx")

churn_cust_info.head()

In [ ]:
churn_acct_info = pd.read_excel("Bank+Customer+Churn/Bank_Churn_Messy.xlsx", sheet_name=1)

churn_acct_info.head()

In [ ]:
churn_df = churn_cust_info.merge(churn_acct_info, how= "left", on = "CustomerId")

churn_df.head()

In [ ]:
churn_df.info()

In [ ]:
churn_df=churn_df.drop("Tenure_y",axis=1).rename({"Tenure_x":"Tenure"},axis=1)

churn_df.head()

In [ ]:
churn_df.duplicated(keep=False)

In [ ]:
churn_df.tail()

In [ ]:
churn_df = churn_df.drop_duplicates()

In [ ]:
churn_df.info()

In [ ]:
#churn_df=churn_df.drop("Tenure_y",axis=1).rename({"Tenure_x":"Tenure"},axis=1).drop_duplicates()
#or we can just do this to drop the duplicates in one step

In [ ]:
churn_df.head()

In [ ]:
#churn_df["EstimatedSalary"].str.strip("€").astype("float")

#we get error because of one negative value

In [ ]:
#so we need to use replace and an empty string

In [ ]:
churn_df["EstimatedSalary"]=churn_df["EstimatedSalary"].str.replace("€","").astype("float")

In [ ]:
churn_df.info()

In [ ]:
churn_df["Balance"]=churn_df["Balance"].str.replace("€","").astype("float")
churn_df.info()

In [ ]:
churn_df[churn_df.isna().any(axis=1) == True]

In [ ]:
churn_df= churn_df.fillna(value={"Surname":"MISSING","Age":churn_df["Age"].median()})

In [ ]:
churn_df.iloc[[28,121,9389]]

In [ ]:
churn_df.describe()

In [ ]:
#there is negative estimated salary of -999999.000000

In [ ]:
#there is negative estimated salary of -999999.000000

In [ ]:
#fixing geography

In [ ]:
churn_df["Geography"].value_counts()

In [ ]:
#we see three different values who are all france

In [ ]:
import numpy as np
churn_df["Geography"]= np.where(churn_df["Geography"].isin(["FRA","France","French"]),"France",churn_df["Geography"])

In [ ]:
churn_df["Geography"].value_counts()

In [ ]:
#we could do the entire thing in one go as below

churn_df= (
    pd.read_excel("Bank+Customer+Churn/Bank_Churn_Messy.xlsx")
    .merge(pd.read_excel("Bank+Customer+Churn/Bank_Churn_Messy.xlsx",sheet_name=1),how="left",on="CustomerId")
    .drop_duplicates()
    .drop("Tenure_y",axis=1)
    .rename({"Tenure_x":"Tenure"},axis=1)
    .assign(
        EstimatedSalary= lambda x:x["EstimatedSalary"].str.replace("€","").astype("float"),
        Balance=lambda x: x["Balance"].str.replace("€","").astype("float"),
        Geography=lambda x: np.where(x["Geography"].isin(["France","French","FRA"]),"France",x["Geography"])
    )
    .assign(EstimatedSalary= lambda x:x["EstimatedSalary"].replace(-999999,x["EstimatedSalary"].median()))
)
churn_df=churn_df.fillna(value={"Surname":"MISSING","Age":churn_df["Age"].median()})

churn_df.head()


In [ ]:
#EDA

In [ ]:
churn_df["Exited"].value_counts(normalize=True).plot.bar()

In [ ]:
import seaborn as sns

sns.barplot(data=churn_df,x="Geography",y="Exited")

In [ ]:
churn_df.select_dtypes("object")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for col in churn_df.drop("Surname",axis=1).select_dtypes("object"):
    sns.barplot(data=churn_df, x=col, y= "Exited")
    plt.show()
    

In [ ]:
#german customers has 2x chrun rate. females higher. no credit cards higher churn
#non active members have higher churn rate

In [ ]:
#lets epxplore nunmeric variables

In [ ]:
for col in churn_df.drop("CustomerId",axis=1).select_dtypes("number"):
    sns.boxplot(data=churn_df,x="Exited",y=col)
    plt.show()

In [ ]:
for col in churn_df.drop("CustomerId",axis=1).select_dtypes("number"):
    sns.histplot(data=churn_df,x=col,hue="Exited",kde=True)
    plt.show()

In [ ]:
#prepare the data for modeling

In [ ]:
modelling_df=churn_df.drop(["CustomerId","Surname"],axis=1)

modelling_df.head()

In [ ]:
#creating dummies

In [ ]:
pd.get_dummies(modelling_df,drop_first=True)

In [ ]:
# modelling_df=pd.get_dummies(modelling_df,drop_first=True)

# we are doing this later

In [ ]:
modelling_df["Balance_v_sal"]=modelling_df["Balance"]/modelling_df["EstimatedSalary"]

modelling_df.head()

In [ ]:
sns.boxplot(data=modelling_df, y="Balance_v_sal")

In [ ]:
#have an extreme outlier. people might not take salary if retired. ratio seems skewed


In [ ]:
modelling_df.describe()

In [ ]:
#very high ratio so we try 

In [ ]:
sns.boxplot(data=modelling_df.query("Balance_v_sal < 10"), y="Balance_v_sal")

In [ ]:
#we can see a lot of outliers but most are between 0 and 1.5

In [ ]:

sns.barplot(data=modelling_df.query("Balance_v_sal < 10"),x="Exited",y="Balance_v_sal", hue="Exited")

In [ ]:
modelling_df.describe()

In [ ]:
modelling_df.to_csv('Bank+Customer+Churn/modelling_df.csv', index=False)


In [ ]:
modelling_df.info()

In [ ]:
#churn_df.to_csv('Bank+Customer+Churn/churn_df.csv', index=False)